In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# 1. Extracting NYT data

### 1.1 Extracting HTML w/ BeautifulSoup

#### Import NYT text file 

This text file is manually generated by copying XML from: https://www.nytimes.com/reviews/dining, reviews starting from 2012. We then use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to format the data for data extraction.

In [143]:
# import nyt html text file (manually generated from: https://www.nytimes.com/reviews/dining)
text = open('import/nyt.txt', 'r')
text = text.read()

# use BeautifulSoup to format
soup = BeautifulSoup(text,"html.parser")

# Extract restaurant review sections
top_rest = soup.find_all('div', attrs={'class': 'story-meta'})

#### Extract links to each NYT review.

In [185]:
links_all = []
links = soup.find_all('div', attrs={'class':'story-body'})
for body in links:
    link = body.find_all('a', attrs={"class":"story-link"})[0].get('href')
    links_all.append(link)

#### Extract names of reviewed restaurants in NYT.

In [186]:
list_rest = []
for tr in top_rest:
    result = tr.find_all('h2', attrs={'class': 'headline'})
    result_name = result[0].string.split()
    result_name = " ".join(result_name)
    list_rest.append(result_name)

In [187]:
print('Number of reviews: ', len(list_rest))

Number of reviews:  590


#### Extract date/time of review, which is contained in a separate XML section.

In [188]:
# Extract HTML sections containing review time/date
top_rest_time = soup.find_all('footer', attrs={'class':'story-footer'})

# Extract date/time
list_time = []
for tr in top_rest_time:
    
    result = tr.find_all('time', attrs={'class': 'dateline'})
    result = result[0]
    list_time.append(result.string)

In [189]:
df_nyt = pd.DataFrame({'nyt_name':list_rest, 'review_time':list_time, 'nyt_link':links_all})
df_nyt.head()

,nyt_link,nyt_name,review_time
0,https://www.nytimes.com/2018/06/07/dining/dave...,Davelle,"June 7, 2018"
1,https://www.nytimes.com/2018/05/31/dining/lahi...,Lahi,"May 31, 2018"
2,https://www.nytimes.com/2018/05/29/dining/don-...,Don Angie,"May 29, 2018"
3,https://www.nytimes.com/2018/05/24/dining/bar-...,Bar Patrón by Rockpool,"May 24, 2018"
4,https://www.nytimes.com/2018/05/24/dining/rang...,Rangoon Spoon,"May 24, 2018"


In [190]:
df_nyt.to_csv('test.csv')

# 2. Yelp API

We use the Python library [YelpAPI](https://github.com/gfairchild/yelpapi). Accessing Yelp's API requires an API key, which is obtained by registering project as an app.

In [191]:
from yelpapi import YelpAPI

api_key = ''
yelp_api = YelpAPI(api_key)

### 2.1 Extract Yelp ID and other basic business info.
Using our generated list of NYT restaurants, conduct a search query to Yelp's API to identify the restaurant's Yelp ID, among other basic info. This ID is used to look up more details in the next step.

In [ ]:
import string

y_id = []
y_isclosed = []
y_name = []
y_price = []
y_rating = []
y_reviewcount = []
y_url = []

for rest in list_rest:
    
    # Conduct business search in Yelp API
    
    rest_punct = rest.translate(string.punctuation) # remove punctuation to avoid confounding Yelp search
    response = yelp_api.search_query(term=rest_punct, location='new york city, ny', sort_by='rating', limit=1) # limit=1 for simplicity; we assume the top 1 result is the actual restaurant
    
    if not response['businesses']: # no search result
        y_id.append(np.nan)
        y_isclosed.append(np.nan)
        y_name.append(rest)
        y_price.append(np.nan)
        y_rating.append(np.nan)
        y_reviewcount.append(np.nan)
        y_url.append(np.nan)
        continue
        
    else:
        response = response['businesses'][0]
        # Append to lists, to combine into dataframe of all restaurants' Yelp data
        y_id.append(response['id'])
        y_isclosed.append(response.get('is_closed',np.nan)) 
        y_name.append(response['name'])
        y_price.append(response.get('price',np.nan))
        y_rating.append(response['rating'])
        y_reviewcount.append(response['review_count']) 
        y_url.append(response['url'])

Combine results to dataframe

In [12]:
df_yelp = pd.DataFrame({
            'id':y_id,
            'yelp_name':y_name,
            'rating':y_rating,
            'reviewcount':y_reviewcount,
            'isclosed':y_isclosed,
            'y_url':y_url,
            'y_price':y_price})

Convert "$" Yelp price level format to numeric string.

In [14]:
df_yelp['y_price'] = df_yelp['y_price'].fillna('#') # fill NaN's in 'yelp_price' column w/ "#"
price_dict = {"$":"1", "$$":"2", "$$$":"3", "$$$$":"4", "#":"0"} 

for idx, row in df_yelp.iterrows():
    price = row['y_price']
    if price == "$":
        price = 1
    if price == "$$":
        price = 2
    if price == "$$$":
        price = 3
    if price == "$$$$":
        price = 4
    df_yelp.loc[idx, 'y_price'] = price

df_yelp.head()

,id,isclosed,rating,reviewcount,y_price,y_url,yelp_name
0,PwrYnl3j3YCGzILX_5UHpw,False,4.0,48.0,2,https://www.yelp.com/biz/davelle-new-york-2?ad...,Davelle
1,GT1OdWlLO9kRQf0SEOrrVA,False,4.0,34.0,2,https://www.yelp.com/biz/lahi-elmhurst?adjust_...,Lahi
2,h37t9rA06Sr4EetJjKrfzw,False,4.5,128.0,3,https://www.yelp.com/biz/don-angie-new-york?ad...,Don Angie
3,NaN,NaN,NaN,NaN,#,NaN,Bar Patrón by Rockpool
4,chdmH_y3YReZgIlNzj8djQ,False,3.5,56.0,2,https://www.yelp.com/biz/rangoon-spoon-brookly...,Rangoon Spoon


Pickle dataframe.

In [ ]:
df_yelp.to_pickle('df_yelp.pkl')

#### Combine all info so far into one dataframe

In [37]:
df_nyt_yelp = pd.concat([df_nyt, df_yelp], axis=1)
df_nyt_yelp.head()

,nyt_name,review_time,id,isclosed,rating,reviewcount,y_price,y_url,yelp_name
0,Davelle,"June 7, 2018",PwrYnl3j3YCGzILX_5UHpw,False,4.0,48.0,2,https://www.yelp.com/biz/davelle-new-york-2?ad...,Davelle
1,Lahi,"May 31, 2018",GT1OdWlLO9kRQf0SEOrrVA,False,4.0,34.0,2,https://www.yelp.com/biz/lahi-elmhurst?adjust_...,Lahi
2,Don Angie,"May 29, 2018",h37t9rA06Sr4EetJjKrfzw,False,4.5,128.0,3,https://www.yelp.com/biz/don-angie-new-york?ad...,Don Angie
3,Bar Patrón by Rockpool,"May 24, 2018",NaN,NaN,NaN,NaN,#,NaN,Bar Patrón by Rockpool
4,Rangoon Spoon,"May 24, 2018",chdmH_y3YReZgIlNzj8djQ,False,3.5,56.0,2,https://www.yelp.com/biz/rangoon-spoon-brookly...,Rangoon Spoon


### 2.2 Fix errors & fill missing data

#### Export data, manually inspect for errors, import back.
During this process, the dates were manually re-formatted to be in the same format as well. Errors usually due to:
1. NYT and Yelp restaurant spelling was different - thus, using NYT's spelling to conduct Yelp API search yielded inaccurate results.
2. Restaurant closed and did not turn up in Yelp API search (but Yelp listing/URL still exists)

In [ ]:
df_nyt_yelp.to_csv('import/df_nyt_yelp.csv')

Edited errors manually in Excel sheet and renamed `df_nyt_yelp2.csv`, to import back into this notebook.

In [164]:
df = pd.read_csv('import/df_nyt_yelp2.csv')

# correct some ID's (beginning with characters that don't carry over from CSV files)
df.loc[35, 'id'] = '-Exv1AEsaKU6Cdb2TO_ZUg'
df.loc[112, 'id'] = '-sUqq_Gty6LTWcOzosjBrQ'
df.loc[116, 'id'] = '-hUvO0C0A-pGZdiSKvjoFw'
df.loc[240, 'id'] = '-ic13XJ9ekpEfshQFPKL4A'
df.loc[309, 'id'] = '-u9yPjmvZilrXYM8JlKG6w'
df.loc[411, 'id'] = '-i0BX3tNEhVPRuhvWxlayQ'
df.loc[428, 'id'] = '-vF3hX7v1R4oAjfNgifZvA'
df.loc[541, 'id'] = '-VvrQWfbkQWeuGC4L0KcfQ'

# replace mistakes w/ re-imported CSV's new corrections
df_nyt_yelp['id'] = df['id']
df_nyt_yelp['isclosed'] = df['isclosed']
df_nyt_yelp['rating'] = df['rating']
df_nyt_yelp['reviewcount'] = df['reviewcount']

# if URL is also being corrected, replace from the CSV as well
correct_url = df[df['correct_url'].isnull() == False]
correct_url_idx = correct_url.index.values

for idx, restaurant in df_nyt_yelp.iterrows():
    if idx in correct_url_idx:
        df_nyt_yelp.loc[idx, 'y_url'] = correct_url.loc[idx, 'correct_url']
        df_nyt_yelp.loc[idx, 'isclosed'] = correct_url.loc[idx, 'isclosed']
        df_nyt_yelp.loc[idx, 'rating'] = correct_url.loc[idx, 'rating']
        df_nyt_yelp.loc[idx, 'reviewcount'] = correct_url.loc[idx, 'reviewcount']
        df_nyt_yelp.loc[idx, 'y_price'] = correct_url.loc[idx, 'y_price']

df_nyt_yelp.head()

,nyt_name,nyt_review_time,yelp_id,isclosed,yelp_rating,reviewcount,yelp_price,yelp_url,yelp_name,critics_pick,nyt_stars,nyt_link,id,rating,y_url,y_price
0,Davelle,2018-06-07,PwrYnl3j3YCGzILX_5UHpw,False,4.0,47.0,2,https://www.yelp.com/biz/davelle-new-york-2,Davelle,y,NaN,https://www.nytimes.com/2018/06/07/dining/dave...,PwrYnl3j3YCGzILX_5UHpw,4.0,NaN,NaN
1,Lahi,2018-05-31,kDqW1eeZEDTitXerwzwCzw,False,4.0,34.0,$$,https://www.yelp.com/biz/lahi-elmhurst,Lahi,n,NaN,https://www.nytimes.com/2018/05/31/dining/lahi...,kDqW1eeZEDTitXerwzwCzw,4.0,https://www.yelp.com/biz/lahi-elmhurst,$$
2,Don Angie,2018-05-29,h37t9rA06Sr4EetJjKrfzw,False,4.5,126.0,3,https://www.yelp.com/biz/don-angie-new-york,Don Angie,y,4.0,https://www.nytimes.com/2018/05/29/dining/don-...,h37t9rA06Sr4EetJjKrfzw,4.5,NaN,NaN
3,Rangoon Spoon,2018-05-24,chdmH_y3YReZgIlNzj8djQ,NaN,3.5,NaN,2,https://www.yelp.com/biz/rangoon-spoon-brooklyn,Rangoon Spoon,n,NaN,https://www.nytimes.com/2018/05/24/dining/rang...,NaN,NaN,NaN,NaN
4,Wokuni,2018-05-22,D6ZEcG1FCZ18nekMWrhgMg,False,4.0,55.0,3,https://www.yelp.com/biz/wokuni-new-york,Wokuni,n,3.0,https://www.nytimes.com/2018/05/24/dining/rang...,chdmH_y3YReZgIlNzj8djQ,3.5,NaN,NaN


#### Delete the establishments without Yelp data. 

This missing info is usually due to:
- Restaurant closed before listing was active on Yelp.
- Restaurant was not located in New York area (other countries, across the US, etc).
- Restaurant was an upstairs/subsection bar/restaurant of another establishment, which has since been defunct.

In [39]:
df_nyt_yelp = df_nyt_yelp.drop(df[df['delete']=='y']['index'])

Rename columns so that origin of data (NYT or Yelp) is clear.

In [41]:
df_nyt_yelp = df_nyt_yelp.rename({'review_time': 'nyt_review_time',
                    'id':'yelp_id',
                    'rating':'yelp_rating',
                    'y_url':'yelp_url',
                    'y_price':'yelp_price',}, axis='columns')
df_nyt_yelp.head()

,nyt_name,nyt_review_time,yelp_id,isclosed,yelp_rating,reviewcount,yelp_price,yelp_url,yelp_name
0,Davelle,"June 7, 2018",PwrYnl3j3YCGzILX_5UHpw,False,4.0,47.0,2,https://www.yelp.com/biz/davelle-new-york-2?ad...,Davelle
1,Lahi,"May 31, 2018",kDqW1eeZEDTitXerwzwCzw,False,4.0,34.0,$$,https://www.yelp.com/biz/lahi-elmhurst,Lahi
2,Don Angie,"May 29, 2018",h37t9rA06Sr4EetJjKrfzw,False,4.5,126.0,3,https://www.yelp.com/biz/don-angie-new-york?ad...,Don Angie
4,Rangoon Spoon,"May 24, 2018",chdmH_y3YReZgIlNzj8djQ,False,3.5,55.0,2,https://www.yelp.com/biz/rangoon-spoon-brookly...,Rangoon Spoon
5,Wokuni,"May 22, 2018",D6ZEcG1FCZ18nekMWrhgMg,False,4.0,60.0,3,https://www.yelp.com/biz/wokuni-new-york?adjus...,Wokuni


#### Append NYT review "stars" (1-5) and "Critic's Pick" (y/n). 

This is done by importing the manually generated `df_nyt_stars.csv`, which was assembled by reviewing info on NYT website.

In [ ]:
df_nyt_stars = pd.read_csv('import/df_nyt_stars.csv')
df_nyt_stars = df_nyt_stars.drop('Unnamed: 0', axis=1)

# Add NYT star info to master dataframe (df_nyt_yelp)
df_nyt_yelp['critics_pick'] = df_nyt_stars['critics_pick'].tolist()
df_nyt_yelp['nyt_stars'] = df_nyt_stars['nyt_stars'].tolist()

# Fill NaN's under "Critic's Pick" column w/ "n" (no)
df_nyt_yelp['critics_pick'] = df_nyt_yelp['critics_pick'].fillna('n') 

df_nyt_stars.head()

Revise NYT stars system, so that string-ratings such as "Poor", "Fair", and "Satisfactory" are converted to a numeric rating. This is accomplished by assigning "Poor" = 0, "Fair" = 1, and "Satisfactory" = 2 and adding 2 to each rating that was originally starred.

In [51]:
new_stars = []
for star in df_nyt_yelp['nyt_stars']:
    if type(star)==str:
        if star not in ['poor','fair','satisfactory']:
            star=int(star)+2
        if star=='poor':
            star=0
        if star=='fair':
            star=1
        if star=='satisfactory':
            star=2
            
    new_stars.append(star)

df_nyt_yelp['nyt_stars'] = new_stars

#### Fill missing URL's

Some restaurants did not receive a URL to fill their row in the `yelp_url` column. This was either because:
1. Their Yelp API response had an anomalous data layout.
2. Their info (except for URL) was manually generated in step 2.2 and URL was left out.

In [52]:
missing_url = df_nyt_yelp[df_nyt_yelp['yelp_url'].isnull()].index.values
print('Number of restaurants missing URLs: ', len(missing_url))

Number of restaurants missing URLs:  8


Since there are only 8 restaurants missing URL's, we proceed with manually correcting the mistakes (saves time, ensures accuracy).

In [53]:
df_nyt_yelp.loc[13, 'yelp_url'] = 'https://www.yelp.com/biz/ludas-dumplings-brooklyn'
df_nyt_yelp.loc[44, 'yelp_url'] = 'https://www.yelp.com/biz/margaritas-restaurant-bronx?osq=margaritas'
df_nyt_yelp.loc[72, 'yelp_url'] = 'https://www.yelp.com/biz/cervos-new-york'
df_nyt_yelp.loc[227, 'yelp_url'] = 'https://www.yelp.com/biz/pravue-cafe-and-albanian-grill-ridgewood'
df_nyt_yelp.loc[280, 'yelp_url'] = 'https://www.yelp.com/biz/hanyang-boonshik-flushing-2'
df_nyt_yelp.loc[404, 'yelp_url'] = 'https://www.yelp.com/biz/gastronomia-culinaria-new-york'
df_nyt_yelp.loc[502, 'yelp_url'] = 'https://www.yelp.com/biz/fishermans-dawta-brooklyn'
df_nyt_yelp.loc[523, 'yelp_url'] = 'https://www.yelp.com/biz/miss-lilys-new-york'
df_nyt_yelp.loc[528, 'yelp_url'] = 'https://www.yelp.com/biz/alison-eighteen-new-york'
df_nyt_yelp.loc[559, 'yelp_url'] = 'https://www.yelp.com/biz/alfama-restaurant-new-york'

# Check that missing URLs are now filled
df_nyt_yelp.loc[missing_url]

# Reset index to account for removed restaurants
df_nyt_yelp = df_nyt_yelp.reset_index()
df_nyt_yelp = df_nyt_yelp.drop('index', axis=1)

#### Fix Yelp URLs

Trim the URLs in `yelp_url` column (i.e. URL to restaurant's Yelp page, provided by Yelp API). Our Yelp webscraper in the next section requires the basic URL that users navigate to when using a desktop computer, not Yelp API's URL - it works for ultimately arriving at business page, but also contains a string of identifiers about how the URL was obtained, etc.

In [54]:
yelp_url_replace = []
for idx, restaurant in df_nyt_yelp.iterrows():
    s = restaurant['yelp_url']
    s = s.split('?')[0] # basic URL is string up to ('?') in Yelp API's URL output
    yelp_url_replace.append(s)

df_nyt_yelp['yelp_url'] = yelp_url_replace

#### Convert date to DateTime object.

Some values in columns are still bs4.elements.Tags. In addition to preventing us from Pickling the dataframe, we should convert to DateTime objects for later analysis.

In [168]:
import re
from dateutil import parser

#calendar_dict = {'Jan.':'01', 'Feb.': '02', 'March':'03', 'April':'04', 'May':'05', 'June':'06',
#                 'July':'07', 'Aug.':'08', 'Sept.':'09', 'Oct.':'10', 'Nov.':'11', 'Dec.':'12'}

times = [str(t) for t in df_nyt_yelp['nyt_review_time']]
times2 = [re.sub(r'[^\w\s]','',t) for t in times]
times3 = [parser.parse(t) for t in times2]

df_nyt_yelp['nyt_review_time'] = times3

#### Convert Yelp ratings to numbers.

Yelp ratings encoded as strings. Convert to numbers.

In [56]:
df_nyt_yelp['yelp_rating'] = pd.to_numeric(df_nyt_yelp['yelp_rating'])

#### Delete Yelp reviews before 2012

Some reviews from 2011 were included in our database - remove those.

In [65]:
delete_2011 = [idx for idx, row in df_nyt_yelp.iterrows() if row['nyt_review_time'].year == 2011]
df_nyt_yelp = df_nyt_yelp.drop(delete_2011)

# 3. Scrape Yelp reviews

Yelp restricts review access on its API and only provides snippet of each review's text. We therefore rely on web scraping to extract review text for each restaurant in our database.

Our web scraping algorithm was adapted from Parkhar Thapak’s code [here](https://www.codementor.io/prakharthapak/scraping-yelp-using-beautiful-soup-gal5m4ts3), which was originally designed to scrape peripheral review information such as votes for “Funny”/”Useful”, number of reviews the user had written, whether the user was “Yelp Elite” status, etc. We added in extraction for review text, date review was written, and rating assigned by Yelp user.

#### Set up a new column in 'df_nyt_yelp' called 'yelp_reviews', which will hold the dictionary of all that restaurant's review data

In [656]:
#df_nyt_yelp['yelp_reviews'] = np.nan
#df_nyt_yelp.head()

,nyt_name,nyt_review_time,yelp_id,isclosed,yelp_rating,reviewcount,yelp_price,yelp_url,yelp_name,critics_pick,nyt_stars,yelp_reviews
0,Davelle,"June 7, 2018",PwrYnl3j3YCGzILX_5UHpw,False,4.0,47.0,2,https://www.yelp.com/biz/davelle-new-york-2,Davelle,y,NaN,NaN
1,Lahi,"May 31, 2018",kDqW1eeZEDTitXerwzwCzw,False,4.5,4.0,1,https://www.yelp.com/biz/lahi-queens,LAHI,n,NaN,NaN
2,Don Angie,"May 29, 2018",h37t9rA06Sr4EetJjKrfzw,False,4.5,126.0,3,https://www.yelp.com/biz/don-angie-new-york,Don Angie,y,2,NaN
3,Rangoon Spoon,"May 24, 2018",chdmH_y3YReZgIlNzj8djQ,False,3.5,55.0,2,https://www.yelp.com/biz/rangoon-spoon-brooklyn,Rangoon Spoon,n,1,NaN
4,Wokuni,"May 22, 2018",D6ZEcG1FCZ18nekMWrhgMg,False,4.0,60.0,3,https://www.yelp.com/biz/wokuni-new-york,Wokuni,y,NaN,NaN


#### Run web scraper

Yelp will sense a scraper, so scraper will fail every so many restaurants. As a result, we have several safe-guards built into our algorithm for collecting all Yelp review data:

1. Pickle each restaurant's review data once it is finished generating. File format is "(restaurant_index).pkl" and is stored in `/import/restaurants`.
2. Pause the code for 1 minute after completing review collection for a restaurant, before initiating collection for next restaurant.
3. Alter loop "df_nyt_yelp.iloc[start:end].iterrows()" to pick up from last failed scraping attempt.

In [ ]:
from urllib.request import urlopen as url
import io
import re
import time

# Create list to store all Yelp review data. 
# Each dict corresponds to a restaurant, where values = each review's info and text for that restaurant.

for idx, restaurant in df_nyt_yelp.iterrows():
    
    print(idx) ##
    print(restaurant) ##
    
    user_count_total = []
    friend_count_total = []
    review_count_total = []
    elite_count_total = []
    funny_count_total = []
    cool_count_total = []
    useful_count_total = []
    length_count_total = []
    reviews_total = []
    rating_total = []
    date_total = []
    rest_dict = {}
    df = pd.DataFrame()
    
    num_pages = int(np.ceil(restaurant['reviewcount'] / 20)) # number of pages of Yelp reviews for this restaurant (to navigate through to scrape all reviews)  

    for _ in range(0,num_pages): 
        
        print(_)
        if _%10==0:
            time.sleep(60)

        my_url= restaurant['yelp_url']+'?start='+str(_*20)
        request=url(my_url)
        htmlscrap=request.read()
        request.close()
        page_soup=BeautifulSoup(htmlscrap,"html.parser")
        container=page_soup.findAll("div",{"class":"review review--with-sidebar"}) # the class name where all the features are contained
        links = page_soup.find_all('div', {'class':'review-list'})

        # Extract reviews 

        for i in container:
            
            if (not i.find_all('span', {'class':'ghost-user'}))==False:
                if i.find_all('span', {'class':'ghost-user'})[0].text[:4]=='Qype':
                    continue
                
            friend_counter=i.findAll("li",{"class":"friend-count responsive-small-display-inline-block"})
            friend_count=friend_counter[0].b.text
            review_counter=i.findAll("li",{"class":"review-count responsive-small-display-inline-block"})
            review_count=review_counter[0].b.text

            elite_counter=i.findAll("li",{"class":"is-elite responsive-small-display-inline-block"})
            if elite_counter:
                elite_count=1
            else:
                elite_count=0
            funny_counter=i.findAll("a",{"class":"ybtn ybtn--small funny js-analytics-click"})
            funny_count1=funny_counter[0].findAll("span",{"class":"count"})
            funny_count=funny_count1[0].text
            if funny_count:
                funny_count=funny_count
            else:
                funny_count=0
            cool_counter=i.findAll("a",{"class":"ybtn ybtn--small cool js-analytics-click"})
            cool_count1=cool_counter[0].findAll("span",{"class":"count"})
            cool_count=cool_count1[0].text
            if cool_count:
                cool_count=cool_count
            else:
                cool_count=0
            useful_counter=i.findAll("a",{"class":"ybtn ybtn--small useful js-analytics-click"})
            useful_count1=useful_counter[0].findAll("span",{"class":"count"})
            useful_count=useful_count1[0].text
            if useful_count:
                useful_count=useful_count
            else:
                useful_count=0
            user_counter=i.findAll("a",{"class":"user-display-name js-analytics-click"})
            user_count=user_counter[0].text
            rating_counter=i.findAll("div",{"class":"biz-rating biz-rating-large clearfix"})
            rating_count=rating_counter[0].div.div["title"]
            rating_count=(int(rating_count[0]))

            length_counter=i.findAll("p",{"lang":"en"})
            xx=str(length_counter[0])
            length_count=len(xx)

            # Extract rating

            rating = i.findAll("div",{"class":'i-stars i-stars--regular-1 rating-large'})
            if not rating:
                rating = i.findAll("div",{"class":'i-stars i-stars--regular-2 rating-large'})
                if not rating:
                    rating = i.findAll("div",{"class":'i-stars i-stars--regular-3 rating-large'})
                    if not rating:
                        rating = i.findAll("div",{"class":'i-stars i-stars--regular-4 rating-large'})
                        if not rating:
                            rating = i.findAll("div",{"class":'i-stars i-stars--regular-5 rating-large'})
            rating = re.search('regular-(\d+)', str(rating[0])).group(1)       

            # Extract date of review

            date_section = i.find_all('span', {'class':'rating-qualifier'})[0]
            date = str(date_section).split()[2]

            # Extract review text

            text = i.find_all('p', {'lang':'en'})[0]
            text_revised = []
            for tag in text:
                if str(type(tag)) == "<class 'bs4.element.NavigableString'>":
                    text_revised.append(tag)
                else:
                    tag = ". "
                    text_revised.append(tag)
            text_revised = ''.join(text_revised)
            text_revised = text_revised.replace(u'\xa0', u' ')

            user_count_total.append(user_count)
            friend_count_total.append(friend_count)
            review_count_total.append(review_count)
            elite_count_total.append(elite_count)
            funny_count_total.append(funny_count)
            cool_count_total.append(cool_count)
            useful_count_total.append(useful_count)
            length_count_total.append(length_count)
            rating_total.append(rating)
            date_total.append(date)
            reviews_total.append(text_revised)

    # Save extracted data, to combine into dataframe
   
    rest_dict = {'user_count':user_count_total,
                 'friend_count':friend_count_total,
                 'review_count':review_count_total,
                 'elite_count':elite_count_total,
                 'funny_count':funny_count_total,
                 'cool_count':cool_count_total,
                 'useful_count':useful_count_total,
                 'length_count':length_count_total,
                 'review_text':reviews_total,
                 'rating':rating_total,
                 'review_date':date_total}
        
    df = pd.DataFrame(rest_dict)
    pkl_name = str(idx) + '.pkl'
    df.to_pickle(pkl_name)
        
    print('done') # Keep track of whether scraper has failed at some point during collection
    time.sleep(60) # pause for 1 min after every query

#### Get the data we have so far

In [6]:
import re
from dateutil import parser

# make a new, second master dataframe that compiles all reviews scraped so far
df_reviews = pd.DataFrame()
rest_length = []
for idx, row in df_nyt_yelp.loc[:334].iterrows():
    pkl_name = 'import/restaurants/' + str(idx) + '.pkl' # get Pickle file name of restaurant
    df = pd.read_pickle(pkl_name) # import pickled dataframe
    df['review_idx'] = idx # affix restaurant's index to dataframe
    df_reviews = df_reviews.append(df) # append to master dataframe of reviews
    rest_length.append(len(df))

df_reviews = df_reviews.reset_index().drop('index', axis=1)

# convert date to DateTime object
df_reviews['review_date'] = [parser.parse(t) for t in df_reviews['review_date']]
#df_reviews['review_date'] = [datetime.strptime(str(t).split()[0], '%Y-%m-%d') for t in df_reviews['review_date']]

# convert rating from str to numeric
df_reviews['rating'] = pd.to_numeric(df_reviews['rating'])

print('Total number of reviews: ', len(df_reviews))
df_reviews.head(2)

Total number of reviews:  91763


,cool_count,elite_count,friend_count,funny_count,length_count,rating,review_count,review_date,review_text,useful_count,user_count,review_idx
0,4,1,45,2,1771,5,345,2018-06-18,"Davelle, uh, oden, uh. Foodie, why you trippin...",4,Jennie C.,0
1,1,1,68,0,791,4,350,2018-07-01,Keep It Simple Smart. Cute all day cafe with ...,1,Yvonne C.,0


Drop restaurants with invalid data, that we didn't catch the first time around (ex. from different country)

In [8]:
# drop invalid resturants

delete_idx = [34,44,47,151,352,498]

if 44 in df_nyt_yelp.index.values: # check if "delete_idx" restaurants already dropped from 'df_nyt_yelp'
    df_nyt_yelp = df_nyt_yelp.drop(delete_idx)

delete_reviews = [idx for idx, row in df_reviews.iterrows() if row['review_idx'] in delete_idx]
df_reviews = df_reviews.drop(delete_reviews)

print('Total number of reviews: ', len(df_reviews))

Total number of reviews:  91763


In [217]:
df_nyt_yelp.head(2)

,nyt_name,nyt_review_time,yelp_id,isclosed,yelp_rating,reviewcount,yelp_price,yelp_url,yelp_name,critics_pick,nyt_stars,nyt_link,id,rating,y_url,y_price
0,Davelle,2018-06-07,PwrYnl3j3YCGzILX_5UHpw,False,4.0,47.0,2,https://www.yelp.com/biz/davelle-new-york-2,Davelle,y,NaN,https://www.nytimes.com/2018/06/07/dining/dave...,PwrYnl3j3YCGzILX_5UHpw,4.0,NaN,NaN
1,Lahi,2018-05-31,kDqW1eeZEDTitXerwzwCzw,False,4.0,34.0,$$,https://www.yelp.com/biz/lahi-elmhurst,Lahi,n,NaN,https://www.nytimes.com/2018/05/31/dining/lahi...,kDqW1eeZEDTitXerwzwCzw,4.0,https://www.yelp.com/biz/lahi-elmhurst,$$


Import link corrections. This was achieved by exporting our `df_nyt_yelp` dataframe up to now (i.e. after dropping restaurants), inspecting their NYT links (under 'nyt_link' column), and correcting.

In [316]:
link_corrections = pd.read_csv('import/nyt_link_corrections.csv')
df_nyt_yelp['nyt_link'] = link_corrections['nyt_link'].tolist()

# 4. Scrape NYT reviews

Scrape NYT reviews to extract text of each review.

#### Scraping #1

In [233]:
from urllib.request import urlopen as url
import io
import re
import time

master_reviewtext = []
for idx, restaurant in df_nyt_yelp.loc[:484].iterrows():
    
    # scrape review page
    my_url= restaurant['nyt_link']
    request=url(my_url)
    htmlscrap=request.read()
    request.close()
    page_soup=BeautifulSoup(htmlscrap,"html.parser")
    container = page_soup.findAll('div', {'class':"css-1h6whtw"})
    
    # extract review text
    review_text = []
    for i, _ in enumerate(container): # reviews are split into paragraphs
        text = _.find_all("p",{"class":"css-1i0edl6 e2kc3sl0"}) # find paragraph
        text_total = []
        for t in text:
            text_total.append(t.text) # concatenate sub-paragraph chunks
        if i == len(container):
            if text_total[-1:][0][:6] == 'Follow': # remove the "Follow NYT on Facebook, Twitter, etc" that concludes each review
                text_total = text_total[:-1]            
        review_text.append(' '.join(text_total)) # join paragraphs into list (review_text)

    master_reviewtext.append(' '.join(review_text))

# append NYT review text to dataframe
df_nyt_yelp['nyt_text'] = master_reviewtext

#### Scraping #2 
Some NYT dining reviews don't have the same format as the first iteration was designed to scrape. Here, we scrape these exceptions with a second format of scraper.

In [376]:
empty_nyt = df_nyt_yelp[df_nyt_yelp['nyt_text']==""].index.values.tolist() # list of restaurant indices w/ empty nyt text

for idx in empty_nyt:
    
    # scrape review page
    restaurant = df_nyt_yelp.loc[idx]
    my_url= restaurant['nyt_link']
    request=url(my_url)
    htmlscrap=request.read()
    request.close()
    page_soup=BeautifulSoup(htmlscrap,"html.parser")
    container = page_soup.findAll('div', {'class':"story-body-supplemental"})
    
    # extract review text
    review_text = []
    for c in container: # review body separated into different chunks of paragraphs
        chunk = c.find_all("p", attrs={"class":"story-body-text story-content"}) # extract the chunk
        for i, _ in enumerate(chunk): # extract each chunk's paragraphs
            subchunk = _.text
            review_text.append(subchunk)           
    
    # append nyt review text to dataframe
    df_nyt_yelp.loc[idx, 'nyt_text'] = ' '.join(review_text)

#### Pickle `df_nyt_yelp` & `df_reviews`

Pickle these dataframes. 
- `df_nyt_yelp` = master database of all our restaurants' Yelp and NYT info. Each row is a restaurant, and each column is information about that restaurant. We will also need to pull this up in our other notebooks to avoid having to re-generate it.
- `df_reviews` = master database of all reviews for these restaurants

In [77]:
df_nyt_yelp.to_pickle('df_nyt_yelp.pkl')
df_reviews.to_pickle('df_reviews.pkl')